# Value an FX Forward using an LSEG-defined curve

This notebook explains how to value an FX Forward using an FX Forward curve defined by LSEG.

## Imports

In [1]:
from lseg_analytics.pricing.market_data import fx_forward_curves
from lseg_analytics.pricing.instruments import fx_forwards
from lseg_analytics.pricing.common import RelativeAdjustableDate, ReferenceDate
import datetime

## Load an FX Forward curve
To load the curve, you need to provide the space where to load the curve from and the name of the curve. This example loads an LSEG-defined curve, so it loads the curve from the 'LSEG' space. You can also load a curve of your own from the 'HOME' space. 

To see the possible curve names, place your cursor after the equals sign and trigger the autocompletion using the _Trigger Suggest_ command (`Ctrl`/`Cmd` + `i`).

In [2]:
# Load the EUR/GBP FX Forward curve from the LSEG storage space
eurgbp_curve = fx_forward_curves.load(space="LSEG", name="EUR_GBP_FxForward")

## Create an FX instrument
To create an FX instrument you need to provide an 'FxForwardDefinition'. At a minimum this requires the 'quoted_currency', 'base_currency', 'payer' and 'receiver', but you can also define the 'start_date', 'end_date', 'rate', 'deal_amount' and 'contra_amount'.

In [3]:
# Create an FX Forward instrument¶
eurgbp_fxfwd = fx_forwards.FxForward(
    definition=fx_forwards.FxForwardDefinition(
        quoted_currency="GBP",
        base_currency="EUR",
        rate=fx_forwards.FxRate(value=0.86),
        end_date=RelativeAdjustableDate(
            tenor="3M", reference_date=ReferenceDate.START_DATE
        ),
        deal_amount=1000000,
        payer="Party1",
        receiver="Party2",
    )
)
eurgbp_fxfwd.save(name="myforward", space="HOME")

True

## Define the market data used to price the FX Forward
Here we need to add the curve to the market_data that will be used to price the FX Forward. 

In [4]:
market_data = fx_forwards.MarketData()
market_data.fx_forward_curves = [
    fx_forwards.FxForwardCurveChoice(reference="EUR_GBP_FxForward")
]

## Value it using the Curve
The 'value' method uses market_data containing the curve which was added in the previous step. In addition to this you can set further parameters including the 'valuation_date' and other preferences. All pricing parameters are optional and will default to a value if not selected. 

In [5]:
valuation_response = eurgbp_fxfwd.value(
    market_data=market_data,
    pricing_preferences=fx_forwards.FxPricingParameters(
        valuation_date=datetime.date(2024, 1, 11),
        report_currency="USD",
    ),
)

## Display the response values
Print the description of the response to see the valuation date, start date and end date in the following format.
```javascript
{'valuationDate': '2024-01-11', 'startDate': {'unAdjusted': '2024-01-16', 'adjusted': '2024-01-16', 'dateMovingConvention': 'NextBusinessDay'}, 'endDate': {'unAdjusted': '2024-04-16', 'adjusted': '2024-04-16', 'dateMovingConvention': 'NextBusinessDay', 'referenceDate': 'StartDate', 'tenor': '3M'}}
```

In addition to the description, the following values can be printed as part of the valuation analysis.

In [6]:
# Display the valuation analysis
print(valuation_response.analytics.description)
print(
    "\t Market value in deal currency:",
    valuation_response.analytics.valuation.market_value_in_deal_ccy,
)
print(
    "\t Market value in contra currency:",
    valuation_response.analytics.valuation.market_value_in_contra_ccy,
)
print(
    "\t Market value in report currency:",
    valuation_response.analytics.valuation.market_value_in_report_ccy,
)
print("\t Greeks - Delta percent:", valuation_response.analytics.greeks.delta_percent)
print(
    "\t Greeks - Delta amount in deal currency:",
    valuation_response.analytics.greeks.delta_amount_in_deal_ccy,
)
print(
    "\t Greeks - Delta amount in contra currency:",
    valuation_response.analytics.greeks.delta_amount_in_contra_ccy,
)

{'valuationDate': '2024-01-11', 'startDate': {'unAdjusted': '2024-01-16', 'adjusted': '2024-01-16', 'dateMovingConvention': 'NextBusinessDay'}, 'endDate': {'unAdjusted': '2024-04-16', 'adjusted': '2024-04-16', 'dateMovingConvention': 'NextBusinessDay', 'referenceDate': 'StartDate', 'tenor': '3M'}}
	 Market value in deal currency: 2991.63229455919
	 Market value in contra currency: 2572.56570294871
	 Market value in report currency: 2013.78535158931
	 Greeks - Delta percent: 99.0133596310335
	 Greeks - Delta amount in deal currency: 987140.328050483
	 Greeks - Delta amount in contra currency: 990133.596310335


In [7]:
fx_forwards.delete(name="myforward", space="HOME")

True